# PaddlePaddle Sentiment Classification with OpenVINO

This simple demo shows how to run bert-base-uncased PaddePaddle model on OpenVINO natively. Instead of exporting the PaddlePaddle model to ONNX and then create the Intermediate Representation (IR) format through OpenVINO optimizer, we can now read direct from the Paddle Model without any conversions.

BERT (Bidirectional Encoder Representations from Transformers) applies Transformers, a popular attention model, to language modelling. This mechanism has an encoder to read the input text and a decoder that produces a prediction for the task. This model uses the technique of masking out some of the words in the input and then condition each word bidirectionally to predict the masked words. BERT also learns to model relationships between sentences, predicts if the sentences are connected or not.

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
from scipy.special import softmax
import urllib.request
import zipfile

from paddlenlp.data import Tuple, Pad
from paddlenlp.transformers import BertTokenizer
from openvino.runtime import Core

## Download PaddlePaddle BERT model

In [ ]:
def callbackfunc(blocknum, blocksize, totalsize):
    '''Callback function to show the download progress
    @blocknum: the number of data block which has been downloaded 
    @blocksize: the size of data blocks
    @totalsize: the total size of a remote file
    '''
    percent = 100.0 * blocknum * blocksize / totalsize
    if percent > 100:
        percent = 100
    sys.stdout.write('Download Progress：{:.5f}%\r'.format(percent))
    sys.stdout.flush()

bert_url = "https://github.com/OpenVINO-dev-contest/models/raw/main/models/bert_sst.zip"
bert_model_path = Path("model/content/PaddleNLP/examples/language_model/bert/infer_model/model.pdmodel")
if bert_model_path.is_file(): 
    print("Model bert-base-uncased already exists")
else:
    # Download the model from the server, and untar it.
    print("Downloading the bert-base-uncased (400Mb)... May take a while...")
    # create a directory 
    os.makedirs("model")
    urllib.request.urlretrieve(bert_url, "model/bert_sst2.zip", callbackfunc)
    print("Model Downloaded")
    try:
        with zipfile.ZipFile("model/bert_sst2.zip") as f:
            f.extractall("./model") 
        print(f"Model Extracted to {bert_model_path}.")
    except Exception as e:
        print("Error Extracting the model, type is:%s" %type(e))
    finally:
        f.close()

## Convert_examples_to_features method
Define the convert_examples_to_features method to get parameters from the input 

In [ ]:
#define the convert_examples_to_features method to get parameters from the input 
def convert_example(text, tokenizer,  max_seq_length=128):
    """
    Define the convert_examples_to_features method to get parameters from the input
    
    :param: text: a list of input sentance
            tokenizer: the pretrained tokenizers object
            max_seq_length: the maximum length of a sentance can be loaded into inference engine, a over-sized sentance will be clustered 
    :retuns:
            input_ids: token embedding
            segment_ids: segment embedding
    """
    encoded_inputs = tokenizer(text=text, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    segment_ids = encoded_inputs["token_type_ids"]

    return input_ids, segment_ids

## Define a predicter with OpenVINO
Here we can read direct from the Paddle Model without any conversions

In [ ]:
def predict(model_path, max_seq_length, data, label_map, batch_size=1):
    """
    Define a predicter with OpenVINO
    
    :param: model_path: the path of .pdmodel
            max_seq_length: the maximum length of a sentance can be loaded into inference engine, a over-sized sentance will be clustered
            data: a list of input sentance
            label_map: a dict of label
            batch_size: batch_size
    :retuns:
            outputs: Negative and Positive probablity
            results: The result label for each sentance
    """
    ie = Core()
    # Directly loading a Paddle format model
    model = ie.read_model(model=model_path + "model.pdmodel")
    compiled_model = ie.compile_model(model=model, device_name="CPU")
    request = compiled_model.create_infer_request()
    output_layer = next(iter(model.outputs))
    examples = []
    # loading (construction and loading) pretrained tokenizers
    tokenizer = BertTokenizer.from_pretrained(
            os.path.dirname(model_path))
    # Convert examples to features
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length)
        examples.append((input_ids, segment_ids))
    # Pads the input data samples to the largest length at `axis`
    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  # input
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  # segment
    ): fn(samples)
    # Seperates data into some batches.
    batches = [
        examples[idx:idx + batch_size]
        for idx in range(0, len(examples), batch_size)
    ]
    outputs = []
    results = []
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        # Do inference, the input data is in dynamic shape
        request.infer(inputs={model.inputs[1].any_name: input_ids, model.inputs[0].any_name: segment_ids})
        result = request.get_output_tensor(output_layer.index).data
        # Postprocessing to get the sentiment label and probility of each sentiment of each sentance
        probs = softmax(result, axis=1)
        idx = np.argmax(probs, axis=1)
        idx = idx.tolist()
        labels = [label_map[i] for i in idx]
        outputs.extend(probs)
        results.extend(labels)
    return outputs, results

## Get the input data and do prediction

The output results are the sentiment probablity of each input sentence

In [ ]:
data = [
    'OpenVINO accelerates applications with high-performance, AI and deep learning inference deployed from edge to cloud',
    'The main disadvantage of padding is a bad performance due to spending time for processing dummy elements in the padding area',
    'Model Optimizer adjusts deep learning models for optimal execution on end-point target devices',
    'Reduce resource demands and efficiently deploy on a range of Intel® platforms from edge to cloud',
    'It may have some accuracy drop'
]
label_map = {0: 'negative', 1: 'positive'}
model_path = "./model/"
max_seq_length = 128
outputs, results = predict(model_path, max_seq_length, data, label_map)
# Print the predicted label and probilitiy of each sentiment
for idx, text in enumerate(data):
    print(
        'Data: {} \n Label: {} \n Negative prob: {} \n Positive prob: {} \n '.
        format(text, results[idx], outputs[idx][0], outputs[idx][1]))